In [1]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
import sqlite3
import time

"""
#Create db
conn = sqlite3.connect('financial_data.db')

# Create a cursor
cursor = conn.cursor()
"""
# Create the first table
#all_time_prices_query = 
'''
CREATE TABLE all_time_prices (
    Date INTEGER PRIMARY KEY,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

#cursor.execute(all_time_prices_query)

"""

momentum_query = '''
CREATE TABLE momentum (
    Date INTEGER PRIMARY KEY,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

cursor.execute(momentum_query)

conn.commit()

# Close the connection
conn.close()
"""

"\n\nmomentum_query = '''\nCREATE TABLE momentum (\n    Date INTEGER PRIMARY KEY,\n    Open FLOAT,\n    High FLOAT, \n    Low FLOAT, \n    Close FLOAT, \n    Volume FLOAT,\n    Dividends FLOAT, \n    Stock Splits FLOAT, \n    Ticker TEXT NOT NULL    \n);\n'''\n\ncursor.execute(momentum_query)\n\nconn.commit()\n\n# Close the connection\nconn.close()\n"

In [7]:
#Create first table with all stock info from yfinance
"""
Working capital has been removed because API calls are too slow. 
Also, working capital is somewhat linked to EBITDA
"""

tick = pd.read_csv("all_stocks")
value_invest_cols = ['Ticker','Price','Trailing PE ratio','Forward PE ratio','Price To Book',\
                                'Price To Sale','EV/EBITDA','EV/GP','Shares to buy']
value_invest = pd.DataFrame(columns = value_invest_cols)
count = 0 
for i in range(len(tick)): 
    start = time.time()    
    print("Producing data for: ", tick["0"][i])
    try:
        ticker = tick["0"][i]
        stock_info = yf.Ticker(ticker)
        d = yf.Ticker(ticker).history(period = "1mo")
        # Get the income statement for the stock

        info = stock_info.info
        fpe = info["forwardPE"]
        tpe = info["trailingPE"]
        fin = stock_info.info
        #liab = stock_info.balance_sheet
        price_to_book = fin['priceToBook']
        price_to_sale = fin['priceToSalesTrailing12Months']
        ev_ebidta = fin['enterpriseToEbitda']
        enterprise_value = fin['enterpriseValue']
        gp = fin['grossProfits']
        ev_gp = enterprise_value/gp
        #current_assets = liab[liab.index=='Current Assets'].iloc[0,0]
        #current_liab = liab[liab.index=='Current Liabilities'].iloc[0,0]
        #working_capital = current_assets/current_liab 

    except TypeError: 
        print("Nonetype found for: " + ticker)
        stop = time.time()
        duration = stop-start
        print(duration)
        continue
    except IndexError: 
        print("Couldn't find: ", ticker)
        stop = time.time()
        duration = stop-start
        print(duration)
        continue
    except KeyError:
        print("Couldnt find key for: " + ticker)
        stop = time.time()
        duration = stop-start
        print(duration)
        continue
    
    d.reset_index(inplace = True)
    d['Ticker'] = ticker
    d["Price"] = d['Close']
    d['Trailing PE ratio'] = tpe
    d['Forward PE ratio'] = fpe
    d['Price To Book'] = price_to_book
    d['Price To Sale'] = price_to_sale
    d['EV/EBITDA'] = ev_ebidta
    d['EV/GP'] = ev_gp
    d = d.iloc[-1]

    #Append to df
    value_invest = value_invest.append(
        pd.Series(d,
        index=value_invest_cols), 
        ignore_index = True
    )

    stop = time.time()
    duration = stop-start
    print(duration)

#Get percentiles for each stock and a specific category
cols = ['Trailing PE ratio', 'Forward PE ratio', 'Price To Book', 'Price To Sale', 'EV/EBITDA', 'EV/GP']
for c in cols: 
    value_invest[c + ' percentile'] = np.NaN
    for index, row in value_invest.iterrows(): 
        percentile_change = stats.percentileofscore(value_invest[c], value_invest[c].loc[index])
        value_invest[c + ' percentile'][index] = percentile_change 

value_invest['RV Score'] = 0

for index, row in value_invest.iterrows(): 
    value_invest['RV Score'][index] = (value_invest['Trailing PE ratio percentile'][index]\
        +value_invest['Forward PE ratio percentile'][index]\
        +value_invest['Price To Book percentile'][index]\
        +value_invest['Price To Sale percentile'][index]\
        +value_invest['EV/EBITDA percentile'][index]\
        +value_invest['EV/GP percentile'][index])/6

#PUSH TO DATABASE
#value_invest.to_sql('all_time_prices', conn, if_exists='append', index=False)

Producing data for:  AOS


<ipython-input-7-ab48f8e95740>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  value_invest = value_invest.append(


1.5872337818145752
Producing data for:  ATEN


<ipython-input-7-ab48f8e95740>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  value_invest = value_invest.append(


1.4116711616516113
Producing data for:  AAC
Nonetype found for: AAC
0.9477014541625977
Producing data for:  AIR


...

<ipython-input-7-ab48f8e95740>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  value_invest = value_invest.append(


1.724942922592163
Producing data for:  AAP


<ipython-input-7-ab48f8e95740>:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  value_invest = value_invest.append(


1.263885736465454
Producing data for:  ADSW
ADSW: No data found, symbol may be delisted


KeyboardInterrupt: 

In [8]:
value_invest

,Ticker,Price,Trailing PE ratio,Forward PE ratio,Price To Book,Price To Sale,EV/EBITDA,EV/GP,Working Capital,Shares to buy
0,AOS,62.580002,19.929935,19.617556,5.470280,2.507204,12.539,6.846755,NaN,NaN
1,ATEN,15.970000,34.717392,19.47561,7.292237,4.290668,18.248,5.205180,NaN,NaN
2,AIR,48.400002,19.206350,13.557424,1.584859,0.904157,11.1,5.535717,NaN,NaN
3,AAN,14.160000,26.222221,8.479042,0.619341,0.207090,4.719,0.978377,NaN,NaN
4,ABB,33.400002,16.292685,21.54839,5.296543,2.145775,14.541,6.898657,NaN,NaN
5,ABT,113.110001,25.532732,25.648527,5.528078,4.379641,14.616,7.999919,NaN,NaN
6,ABBV,157.130005,20.950666,13.384157,17.373951,4.806054,11.653,8.931407,NaN,NaN
7,ANF,27.230000,66.414635,16.112425,2.064756,0.364638,9.902,0.953369,NaN,NaN
8,JEQ,5.480000,4.120301,None,0.777526,33.624226,None,36.844207,NaN,NaN
9,ABM,45.910000,13.307246,11.593434,1.753495,0.385793,9.283,3.866278,NaN,NaN
